# Computational Theory: Analysis of the Countdown Numbers Game

*Jakub Prochnicki G00373793*

---

# Table of Contents
1. [Introduction](#intro)
2. [Explanation of the Countdown Numbers Game](#concepts)
3. [Analysis of the Computational Complexity](#oracle)
4. [Approach to Solving the Game](#implementation)
5. [Conclusion](#conclusion)

# Introduction
---

This notebook constitutes my assessment submission for the Computational Theory Module at ATU Galway. The aim of this notebook is to provide an analysis of the Countdown Numbers Game. Firstly, I will provide an explanation of the game and give an example of the game in action. Following the explanation, I delve into an analysis of the computational complexity of the game and provide various approaches to solving the game, namely: Brute Force Approach, Heuristic Approach and Reverse Polish Notation. Lastly, I will evaluate my findings from solving the problem using various approaches

# Explanation of the Countdown Numbers Game
---
The Countdown Numbers Game allows people to showcase their math skills on a British aired TV Show, Countdown. The aim of the game is to reach a specific target number using a set of six other numbers and basic arithmetic operations. In order to better understand the game, I have come up with the following breakdown of how a round of Countdown works.

- The player selects six numbers from two groups : The first group consisting of four large numbers: 25, 50, 75, and 100. The other group holds twenty smaller numbers ranging from 1 to 10, with each number appearing twice. The player has the freedom to use any combination of numbers from these two groups.

- The game generates a target number, a three digit number.
- The player is then given a 30 second window and is given the task of using a combination of arithmetic operations in order to come as close as possible to the target number. Each of the six numbers can be used once and players are not obliged to use all six numbers. The operations must however result in a whole number at every step.
- The score is determined by the closeness of the player's number to the target score. A perfect score of 10 is awarded to players where they have reached the target number.

  *need to give example of a game in action*

# Analysing the Computational Complexity of the Game
---

*The Problem:* In the game, a player is given six numbers and must use arithmetic operations (addition, subtraction, multiplication, and division) to reach a randomly generated three-digit target number. The operations must result in whole numbers, and each of the six numbers can be used at most once.


In order to analyse the computational complexity, it is necessary to consider how difficult it is to find a solution to reach the target number using a set of numbers and arithmetic operations. This analysis typically focuses on the theoretical limits of solving this problem using algorithms, and the complexity can be described in terms of both time and space (memory). 

The primary challenge is the *combinatorial nature* of the problem. For each of the six numbers, there are decisions about:

 - Whether to use the number or not.
 - Which arithmetic operation to apply.
 - Which other number to combine it with.

As there are four operations and up to five other numbers to combine with, the possibilities increase exponentially with each step.
 
*Permutations:* The numbers can be arranged in 6 different ways, resulting in a factorial of 6

*Operations:* At each step between two numbers, you can apply one of four operations. If you consider operations between pairs of numbers, the number of operational combinations also increases rapidly.


# Implementing Countdown using a Brute Force Approach
---

The Brute Force Approach is the first approach to solving the game that I demonstrate in this notebook. In this approach, you must generate all the  possible number and operation combinations and evaluate each combination to see if it can produce the target number. 

An implementation of the countdown game using this approach is demonstrated using the code below which was generated by ChatGPT using the following prompt:

```
Implement the countdown numbers game in python using a brute force approach.
```


In [4]:
import itertools
import operator

def eval_expression(expr):
    """ Evaluate a traditional infix expression using a stack-based approach for operators and operands. """
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    stack = []
    try:
        for token in expr:
            if token in ops:
                b, a = stack.pop(), stack.pop()
                if token == '/' and b == 0:
                    return None  # Avoid division by zero
                if token == '-' and a - b < 0:
                    return None  # Ensure subtraction doesn't go negative
                if token == '/' and a % b != 0:
                    return None  # Ensure divisions are integral
                stack.append(ops[token](a, b))
            else:
                stack.append(token)
        return stack[0] if len(stack) == 1 else None
    except Exception:
        return None

This function evaluates an expression given in infix notation (like 2 + 3 * 4). It handles basic arithmetic operations and ensures operations like division by zero or negative results from subtraction are not considered valid.

In [5]:
def generate_expressions(numbers, operations):
    """ Generate all valid expressions combining numbers and operations. """
    for ops_perm in itertools.permutations(operations, len(numbers) - 1):
        # Create a pattern like [num, op, num, op, num, ...]
        expression = []
        nums = iter(numbers)
        expression.append(next(nums))
        for op in ops_perm:
            expression.append(op)
            expression.append(next(nums))
        yield expression

 This generates all permutations of inserting operations between the numbers. This function ensures that every possible combination of the given numbers with the given operations in all possible orders is considered.

In [6]:
def solve_numbers(numbers, target):
    """ Brute-force search to find an expression that evaluates to the target number. """
    # Define the operations available
    operations = ['+', '-', '*', '/'] * (len(numbers) - 1)
    for expr in generate_expressions(numbers, operations):
        if eval_expression(expr) == target:
            return expr  # Return the successful expression
    return None

This is the main function that orchestrates the process. It tries every generated expression and checks if it evaluates to the target number. If such an expression is found, it returns it.

In [7]:
import time

# Example usage
numbers = [25, 50, 3, 7, 8, 2]
target = 752

start_time = time.time()
solution = solve_numbers(numbers, target)
elapsed_time = time.time() - start_time

if solution:
    print("Found a solution:", ' '.join(map(str, solution)))
else:
    print("No solution found")
print(f"Elapsed time: {elapsed_time} seconds")


No solution found
Elapsed time: 4.486725807189941 seconds


Here, we can see an example usage of the game using this approach. The complexity of this apporoach is measured by calculating the execution time using the time module.

This implementation solves the problem by creating all possible permutations of operations and numbers. It then checks each permutation to see if it is equal to the target number. Using this approach ensures that a solution will be found if it exists however it comes with the cost of a high computational complexity.

# Reverse Polish Notation
---

Reverse Polish Notation (RPN), also known as postfix notation is a mathematical notation invented in the 1950s that places the arithmetic operator after the arguments it is being operated on. In RPN there is no need for brackets to indicate the grouping of the terms, instead the expression is evaluated from left to right. This simplifies the computation of the expression which is highly favourable in the problem of solving the Countdown numbers game.(https://mathworld.wolfram.com/ReversePolishNotation.html) 

The figure below depicts the stack structure of an RPN expression, where we can see the following actions being performed: 

 - Push the next appearing number to the stack.
 - Pop the number of the stack twice where an operator appears and push the result of the  operation.

   ADDING PICTURE OF RPN STACK HERE

Using RPN in the context of the Countdown game is beneficial in reducing the computational complexity of the game as it simplifies the process of mathematical calculations

The following steps must be taken in order to implement the Countdown numbers game using Reverse Polish Notation: 

  1. **Generate valid RPN expressions**: This step involves confirming that the RPN expressions are valid by ensuring that the count of numbers is always greater than the count of operations up to that point, at every point in the sequence.
     
  2. **Evaluate RPN expressions**: In this step, an RPN stack that I explained previously is used in order to evaluate the expression. If the result of the expression matches the target number, the expression is a solution.
     
  3. **Generate Combinations**: Handle the generation of all possible number combinations.


# References 
--- 
[1] Lynch, J. and Yan Weng. “The Computational Complexity of Finding Arithmetic Expressions With and Without Parentheses.” https://www.semanticscholar.org/reader/21f455ca0bbf9a355611bc0593dd1cf8a8d32584

“Countdown” numbers game: the lowdown https://www.daitx.com/2016/05/01/countdown-math/

Reverse Polish Notation: https://mathworld.wolfram.com/ReversePolishNotation.html
